# library 설치+ 데이터 살펴보기

In [1]:
# Standard library imports
import os 
import glob
import numpy as np
import itertools

# Third party imports
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
import lightgbm as LGB
from sklearn.cross_decomposition import PLSRegression
from sklearn.linear_model import Lasso,ElasticNet,Ridge
from sklearn.svm import SVR

from tqdm import tqdm
import plotly 
import plotly.express as px
import plotly.graph_objects as go 
from plotly.subplots import make_subplots
import matplotlib
from matplotlib import font_manager, rc
import shap



In [2]:
df_train=pd.read_csv("train.csv")
df_train.head(3)

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
0,C2483,900,아파트,경상북도,국민임대,39.72,134,38.0,A,15667000,103680,0.0,3.0,1425.0,1015.0
1,C2483,900,아파트,경상북도,국민임대,39.72,15,38.0,A,15667000,103680,0.0,3.0,1425.0,1015.0
2,C2483,900,아파트,경상북도,국민임대,51.93,385,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0


In [319]:
df_test=pd.read_csv("test.csv")
df_test.head(3)

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수
0,C1072,754,아파트,경기도,국민임대,39.79,116,14.0,H,22830000,189840,0.0,2.0,683.0
1,C1072,754,아파트,경기도,국민임대,46.81,30,14.0,A,36048000,249930,0.0,2.0,683.0
2,C1072,754,아파트,경기도,국민임대,46.90,112,14.0,H,36048000,249930,0.0,2.0,683.0


In [4]:
display(df_train.describe())
display(df_test.describe())

,총세대수,전용면적,전용면적별세대수,공가수,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
count,2952.000000,2952.000000,2952.000000,2952.000000,2741.000000,2948.000000,2952.000000,2952.000000
mean,886.661247,44.757215,102.747967,12.921070,0.176578,3.695726,601.668360,559.768293
std,513.540168,31.874280,132.640159,10.778831,0.427408,2.644665,396.407072,433.375027
min,26.000000,12.620000,1.000000,0.000000,0.000000,0.000000,13.000000,13.000000
25%,513.500000,32.100000,14.000000,4.000000,0.000000,2.000000,279.250000,220.000000
50%,779.000000,39.930000,60.000000,11.000000,0.000000,3.000000,517.000000,487.000000
75%,1106.000000,51.562500,144.000000,20.000000,0.000000,4.000000,823.000000,770.000000
max,2568.000000,583.400000,1865.000000,55.000000,3.000000,20.000000,1798.000000,2550.000000


,총세대수,전용면적,전용면적별세대수,공가수,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수
count,1022.000000,1022.000000,1022.000000,1022.000000,980.000000,1022.000000,1022.000000
mean,862.080235,43.706311,100.414873,15.544031,0.136735,4.626223,548.771037
std,536.340894,35.890759,125.997855,11.070140,0.435500,5.414568,342.636703
min,75.000000,9.960000,1.000000,0.000000,0.000000,1.000000,29.000000
25%,488.000000,33.135000,14.000000,6.000000,0.000000,2.000000,286.000000
50%,745.000000,39.720000,60.000000,15.000000,0.000000,3.000000,458.000000
75%,1161.000000,47.400000,140.000000,23.000000,0.000000,5.000000,711.000000
max,2572.000000,583.400000,1341.000000,45.000000,2.000000,50.000000,1696.000000


In [5]:
train_tmp=df_train.iloc[:,0:-1]
train_tmp['type']='train'
test_tmp=df_test
test_tmp['type']='test'

In [6]:
df_all=pd.concat([train_tmp,test_tmp],axis=0) # 행으로 합치기

## train, test 명목 변수 차집합

In [7]:
columns=['임대건물구분','지역','공급유형','자격유형']
for col in columns:
  complement=list(set(df_train[col].unique())-set(df_test[col].unique()))
  print(f"Train 데이터 기준 차집합 {col}:{complement}")

Train 데이터 기준 차집합 임대건물구분:[]
Train 데이터 기준 차집합 지역:['서울특별시']
Train 데이터 기준 차집합 공급유형:['장기전세', '공공임대(5년)', '공공분양']
Train 데이터 기준 차집합 자격유형:['B', 'O', 'F']


train데이터만 갖고있는 명목변수 어떻게 할지?

In [320]:
df_train.rename(columns={"도보 10분거리 내 지하철역 수(환승노선 수 반영)" : "지하철", "도보 10분거리 내 버스정류장 수" : "버스"}, inplace=True)
df_test.rename(columns={"도보 10분거리 내 지하철역 수(환승노선 수 반영)" : "지하철", "도보 10분거리 내 버스정류장 수" : "버스"}, inplace=True)


## 결측치 구하기

In [9]:
df_train.isnull().sum()

단지코드          0
총세대수          0
임대건물구분        0
지역            0
공급유형          0
전용면적          0
전용면적별세대수      0
공가수           0
자격유형          0
임대보증금       569
임대료         569
지하철         211
버스            4
단지내주차면수       0
등록차량수         0
dtype: int64

## 임대건물구분, 지역, 공급유형, 자격유형의 종류 알아보기

In [10]:
def func(dataset):
  columns=['임대건물구분','지역','공급유형','자격유형']
  for i in columns:
    temp=dataset[i].unique()
    print(f"{i} 의 종류:{temp} ")

In [11]:
func(df_train)

임대건물구분 의 종류:['아파트' '상가'] 
지역 의 종류:['경상북도' '경상남도' '대전광역시' '경기도' '전라북도' '강원도' '광주광역시' '충청남도' '부산광역시' '제주특별자치도'
 '울산광역시' '충청북도' '전라남도' '대구광역시' '서울특별시' '세종특별자치시'] 
공급유형 의 종류:['국민임대' '공공임대(50년)' '영구임대' '임대상가' '공공임대(10년)' '공공임대(분납)' '장기전세' '공공분양'
 '행복주택' '공공임대(5년)'] 
자격유형 의 종류:['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O'] 


In [12]:
## 임대보증금이 na인 경우 명목변수의 종류
func(df_train[df_train['임대보증금'].isnull()]) 

임대건물구분 의 종류:['상가' '아파트'] 
지역 의 종류:['강원도' '충청남도' '경상남도' '대전광역시' '부산광역시' '제주특별자치도'] 
공급유형 의 종류:['임대상가' '공공분양'] 
자격유형 의 종류:['D'] 


In [13]:
df_train[df_train['임대보증금'].isnull()!=df_train['임대료'].isnull()]
##임대보증금과 임대료는 동시에 na값을 가짐. 

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,등록차량수


In [14]:
df_test.isnull().sum()

단지코드          0
총세대수          0
임대건물구분        0
지역            0
공급유형          0
전용면적          0
전용면적별세대수      0
공가수           0
자격유형          2
임대보증금       180
임대료         180
지하철          42
버스            0
단지내주차면수       0
type          0
dtype: int64

In [15]:
df_train[df_train['단지코드']=='C1649']

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,등록차량수
2315,C1649,575,아파트,경상남도,공공임대(10년),74.97,80,15.0,A,46000000,456000,NaN,NaN,1066.0,855.0
2316,C1649,575,아파트,경상남도,공공임대(10년),84.95,124,15.0,A,57000000,462000,NaN,NaN,1066.0,855.0
2317,C1649,575,아파트,경상남도,공공임대(10년),84.96,289,15.0,A,57000000,462000,NaN,NaN,1066.0,855.0
2318,C1649,575,아파트,경상남도,공공임대(10년),84.98,82,15.0,A,57000000,462000,NaN,NaN,1066.0,855.0


In [16]:
## 전용면적 이외의 모든 변수값이 같은 행들은 어떻게 처리할 것인가?? 모두 병합한다기에는 전용면적 변수가 의미 없게 됨. 

In [17]:
df_train[df_train['지하철'].isnull()].sample(10)

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,등록차량수
805,C2156,306,아파트,충청남도,국민임대,36.55,120,0.0,A,8670000,80500,NaN,1.0,246.0,204.0
790,C1875,1003,상가,충청남도,임대상가,21.46,1,5.0,D,NaN,NaN,NaN,8.0,192.0,209.0
207,C1704,639,상가,대전광역시,임대상가,31.84,1,34.0,D,NaN,NaN,NaN,7.0,162.0,131.0
107,C1874,619,상가,충청남도,임대상가,23.25,1,2.0,D,NaN,NaN,NaN,2.0,97.0,62.0
121,C1874,619,상가,충청남도,임대상가,34.80,1,2.0,D,NaN,NaN,NaN,2.0,97.0,62.0
803,C1875,1003,상가,충청남도,임대상가,50.08,1,5.0,D,NaN,NaN,NaN,8.0,192.0,209.0
102,C1874,619,상가,충청남도,임대상가,17.40,1,2.0,D,NaN,NaN,NaN,2.0,97.0,62.0
195,C1616,1507,상가,대전광역시,임대상가,75.98,1,1.0,D,NaN,NaN,NaN,2.0,407.0,98.0
99,C1874,619,아파트,충청남도,영구임대,26.37,149,2.0,C,3141000,69900,NaN,2.0,97.0,62.0
216,C2258,965,아파트,대전광역시,영구임대,31.32,180,13.0,C,6873000,94990,NaN,3.0,287.0,78.0


In [18]:
df_train.groupby('지역')[['지하철','버스']].describe().loc[['충청남도','대전광역시','경상남도']]

지하철                                                  버스            \
       count      mean       std  min  25%  50%  75%  max  count      mean   
지역                                                                           
충청남도    44.0  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0  158.0  3.677215   
대전광역시  173.0  0.722543  0.449043  0.0  0.0  1.0  1.0  1.0  266.0  4.714286   
경상남도   365.0  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0  365.0  3.824658   

                                           
            std  min  25%  50%  75%   max  
지역                                         
충청남도   2.824526  1.0  2.0  2.0  4.0  12.0  
대전광역시  2.872305  1.0  2.0  4.0  6.0  12.0  
경상남도   3.500698  1.0  2.0  3.0  4.0  20.0

## 지하철, 버스는 지역적 조건에 의해 영향을 많이 받으므로, 지역별 평균값으로 na값을 채우는 것을 생각해볼 수 있음. 

In [19]:
df_train[df_train['지하철'].isnull()]['지역'].unique()


array(['충청남도', '대전광역시', '경상남도'], dtype=object)

In [20]:
df_train[df_train['버스'].isnull()]['지역'].unique()

array(['경상남도'], dtype=object)

In [324]:
df_test[df_test['지하철'].isnull()]['지역'].unique()

array(['충청남도', '대전광역시'], dtype=object)

지하철의 경우 :충남->0,대전->0.7,경남->0으로 대체,
버스의 경우 : 경남->3.8로 대체.

In [21]:
df_train[df_train['임대료'].isnull()]['공급유형'].unique()

array(['임대상가', '공공분양'], dtype=object)

In [22]:
df_train['공급유형'].unique()

array(['국민임대', '공공임대(50년)', '영구임대', '임대상가', '공공임대(10년)', '공공임대(분납)',
       '장기전세', '공공분양', '행복주택', '공공임대(5년)'], dtype=object)

In [23]:
df_train.groupby('공급유형')['자격유형'].value_counts()

공급유형       자격유형
공공분양       D          7
공공임대(10년)  A        205
공공임대(50년)  A         31
공공임대(5년)   A          3
공공임대(분납)   A         12
국민임대       A       1539
           H        155
           E         34
           B         21
           G          9
영구임대       C         95
           I         49
           E          3
           F          3
           A          2
임대상가       D        562
장기전세       A          9
행복주택       J        114
           K         33
           L         33
           N         30
           M          2
           O          1
Name: 자격유형, dtype: int64

In [24]:
df_train['임대보증금']

0       15667000
1       15667000
2       27304000
3       27304000
4       27304000
          ...   
2947    11346000
2948    14005000
2949    14005000
2950    14005000
2951    14830000
Name: 임대보증금, Length: 2952, dtype: object

In [25]:
df_test.head(10)

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,type
0,C1072,754,아파트,경기도,국민임대,39.79,116,14.0,H,22830000,189840,0.0,2.0,683.0,test
1,C1072,754,아파트,경기도,국민임대,46.81,30,14.0,A,36048000,249930,0.0,2.0,683.0,test
2,C1072,754,아파트,경기도,국민임대,46.90,112,14.0,H,36048000,249930,0.0,2.0,683.0,test
3,C1072,754,아파트,경기도,국민임대,46.90,120,14.0,H,36048000,249930,0.0,2.0,683.0,test
4,C1072,754,아파트,경기도,국민임대,51.46,60,14.0,H,43497000,296780,0.0,2.0,683.0,test
5,C1072,754,아파트,경기도,국민임대,51.71,51,14.0,H,43497000,296780,0.0,2.0,683.0,test
6,C1072,754,아파트,경기도,국민임대,51.96,198,14.0,H,43497000,296780,0.0,2.0,683.0,test
7,C1072,754,아파트,경기도,국민임대,51.96,67,14.0,H,43497000,296780,0.0,2.0,683.0,test
8,C1128,1354,아파트,경기도,국민임대,39.79,368,9.0,H,22830000,189840,0.0,3.0,1216.0,test
9,C1128,1354,아파트,경기도,국민임대,39.79,30,9.0,H,22830000,189840,0.0,3.0,1216.0,test


In [26]:
df_test.groupby('단지코드')['단지내주차면수']

# 전처리

## 임대보증금, 임대료 공란  0으로 처리

In [332]:
temp_train=df_train.copy()
temp_train[['임대보증금','임대료']] = temp_train[['임대보증금', '임대료']].fillna("0").replace("-", "0").astype(int)
temp_test=df_train.copy()
temp_test[['임대보증금','임대료']] = temp_test[['임대보증금', '임대료']].fillna("0").replace("-", "0").astype(int)



## 지하철, 버스  공란 지역별 평균으로 처리

In [333]:
temp_train.groupby('지역')[['지하철','버스']].describe().loc[['충청남도','대전광역시','경상남도']]

지하철                                                  버스            \
       count      mean       std  min  25%  50%  75%  max  count      mean   
지역                                                                           
충청남도    44.0  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0  158.0  3.677215   
대전광역시  173.0  0.722543  0.449043  0.0  0.0  1.0  1.0  1.0  266.0  4.714286   
경상남도   365.0  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0  365.0  3.824658   

                                           
            std  min  25%  50%  75%   max  
지역                                         
충청남도   2.824526  1.0  2.0  2.0  4.0  12.0  
대전광역시  2.872305  1.0  2.0  4.0  6.0  12.0  
경상남도   3.500698  1.0  2.0  3.0  4.0  20.0

In [334]:
temp_train['지하철'] = temp_train['지하철'].fillna(temp_train.groupby('지역')['지하철'].transform('mean'))
temp_train['버스']=temp_train['버스'].fillna(temp_train.groupby('지역')['버스'].transform('mean'))
temp_test['지하철']=temp_test['지하철'].fillna(temp_test.groupby('지역')['지하철'].transform('mean'))
temp_test['버스']=temp_test['버스'].fillna(temp_test.groupby('지역')['버스'].transform('mean'))

평균값으로 채워진 것을 볼 수 있음

In [335]:
temp_train[df_train['지하철'].isnull()]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,등록차량수
94,C1312,518,아파트,충청남도,국민임대,39.72,60,12.0,A,17460000,122210,0.000000,3.0,527.0,359.0
95,C1312,518,아파트,충청남도,국민임대,39.98,89,12.0,A,17460000,122210,0.000000,3.0,527.0,359.0
96,C1312,518,아파트,충청남도,국민임대,41.55,225,12.0,A,19954000,130940,0.000000,3.0,527.0,359.0
97,C1312,518,아파트,충청남도,국민임대,46.90,143,12.0,A,28687000,149660,0.000000,3.0,527.0,359.0
98,C1874,619,아파트,충청남도,영구임대,26.37,294,2.0,C,3141000,69900,0.000000,2.0,97.0,62.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2339,C1350,1401,아파트,대전광역시,공공임대(10년),59.91,13,2.0,A,37474000,519350,0.722543,6.0,1636.0,2315.0
2340,C1350,1401,아파트,대전광역시,공공임대(10년),59.92,223,2.0,A,37385000,516130,0.722543,6.0,1636.0,2315.0
2341,C1350,1401,아파트,대전광역시,공공임대(분납),51.99,146,2.0,A,54612000,131130,0.722543,6.0,1636.0,2315.0
2342,C1350,1401,아파트,대전광역시,공공임대(분납),59.91,32,2.0,A,63585000,152680,0.722543,6.0,1636.0,2315.0


In [336]:
temp_train.isnull().sum()

단지코드        0
총세대수        0
임대건물구분      0
지역          0
공급유형        0
전용면적        0
전용면적별세대수    0
공가수         0
자격유형        0
임대보증금       0
임대료         0
지하철         0
버스          0
단지내주차면수     0
등록차량수       0
dtype: int64

In [337]:
temp_test.isnull().sum()

단지코드        0
총세대수        0
임대건물구분      0
지역          0
공급유형        0
전용면적        0
전용면적별세대수    0
공가수         0
자격유형        0
임대보증금       0
임대료         0
지하철         0
버스          0
단지내주차면수     0
등록차량수       0
dtype: int64

## 명목변수 범주화

전용면적 10 단위로 범주화함.\
새로운 변수: 임대용총전용면적, key(임대건물구분, 공급유형, 자격유형)\
임대건물구분, 지역, 공급유형,자격유형 범주화함.

In [338]:
(temp_train['전용면적'] //5*5).value_counts()

35.0     667
45.0     644
50.0     388
25.0     324
30.0     263
55.0     203
20.0     156
80.0      80
15.0      74
70.0      48
40.0      41
75.0      11
65.0       9
60.0       8
10.0       8
240.0      6
125.0      5
580.0      5
135.0      4
400.0      3
245.0      2
315.0      1
105.0      1
405.0      1
Name: 전용면적, dtype: int64

In [339]:
(temp_train['전용면적'] //10*10).value_counts()

30.0     930
40.0     685
50.0     591
20.0     480
10.0      82
80.0      80
70.0      59
60.0      17
240.0      8
580.0      5
120.0      5
130.0      4
400.0      4
310.0      1
100.0      1
Name: 전용면적, dtype: int64

In [340]:
temp_train['전용면적'] = temp_train['전용면적'] //10*10
temp_train['전용면적'] = np.where(temp_train['전용면적'] > 100, 100, temp_train['전용면적'])
temp_train['전용면적'] = np.where(temp_train['전용면적'] < 15, 15, temp_train['전용면적'])
temp_train['전용면적'].value_counts()

30.0     930
40.0     685
50.0     591
20.0     480
15.0      82
80.0      80
70.0      59
100.0     28
60.0      17
Name: 전용면적, dtype: int64

In [341]:
temp_test['전용면적'] = temp_test['전용면적'] //10*10
temp_test['전용면적'] = np.where(temp_test['전용면적'] > 100, 100, temp_test['전용면적'])
temp_test['전용면적'] = np.where(temp_test['전용면적'] < 15, 15, temp_test['전용면적'])
temp_test['전용면적'].value_counts()

30.0     930
40.0     685
50.0     591
20.0     480
15.0      82
80.0      80
70.0      59
100.0     28
60.0      17
Name: 전용면적, dtype: int64

In [342]:
temp_train

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,등록차량수
0,C2483,900,아파트,경상북도,국민임대,30.0,134,38.0,A,15667000,103680,0.0,3.0,1425.0,1015.0
1,C2483,900,아파트,경상북도,국민임대,30.0,15,38.0,A,15667000,103680,0.0,3.0,1425.0,1015.0
2,C2483,900,아파트,경상북도,국민임대,50.0,385,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
3,C2483,900,아파트,경상북도,국민임대,50.0,15,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
4,C2483,900,아파트,경상북도,국민임대,50.0,41,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2947,C2532,239,아파트,강원도,국민임대,40.0,19,7.0,A,11346000,116090,0.0,1.0,166.0,146.0
2948,C2532,239,아파트,강원도,국민임대,50.0,34,7.0,A,14005000,142310,0.0,1.0,166.0,146.0
2949,C2532,239,아파트,강원도,국민임대,50.0,34,7.0,A,14005000,142310,0.0,1.0,166.0,146.0
2950,C2532,239,아파트,강원도,국민임대,50.0,114,7.0,A,14005000,142310,0.0,1.0,166.0,146.0


In [343]:
#train데이터에만 있는 명목명수 value값 처리
# option=0;none, option=1:'이외'로 처리, option=2: drop하기

def df(dataset,option):
    train_data=dataset.copy()
    if (option==1):
        train_data.loc[train_data.지역.isin(['서울특별시']), '지역'] = '이외'
        train_data.loc[train_data.공급유형.isin(['공공임대(5년)', '장기전세', '공공분양']), '공급유형'] = '이외'
        train_data.loc[train_data.자격유형.isin(['O', 'B', 'F']), '공급유형'] = '이외'
        
        
    if(option==2):
        train_data = train_data.loc[train_data.지역 != '서울특별시',:]
        train_data = train_data.loc[train_data.공급유형!='공공분양',:]
        train_data = train_data.loc[train_data.공급유형!='장기전세',:]
        train_data = train_data.loc[train_data.공급유형!='공공임대(5년)',:]
        train_data = train_data.loc[train_data.자격유형!='F',:]
        train_data = train_data.loc[train_data.자격유형!='B',:]
        train_data = train_data.loc[train_data.자격유형!='O',:]

    
    
        
    return train_data

In [344]:
#범주화
def categorize(dataset):
    train_data=dataset.copy()
    train_data.loc[:,'임대용총전용면적'] = train_data.loc[:,'전용면적'] * train_data.loc[:,'전용면적별세대수']
    train_data.loc[:,"임대건물구분"] = train_data.loc[:,"임대건물구분"].astype('category').cat.codes
    train_data.loc[:,'지역'] = train_data.loc[:,'지역'].astype('category').cat.codes
    train_data.loc[:,'공급유형'] = train_data.loc[:,'공급유형'].astype('category').cat.codes
    train_data.loc[:,'자격유형'] = train_data.loc[:,'자격유형'].astype('category').cat.codes
    train_data['key'] = train_data['임대건물구분'].astype(str).str.cat(train_data['공급유형'].astype(str), sep='-').str.cat(train_data['자격유형'].astype(str), sep='-')
    
    return train_data

In [345]:
train0=categorize(temp_train)
train0.shape

(2952, 17)

In [346]:
train1=df(temp_train,1)
train1=categorize(train1)

In [347]:
train2=df(temp_train,2)
train2=categorize(train2)
train2.shape

(2853, 17)

## 면적 처리 & 코드 그룹

In [348]:
df_gender=pd.read_csv("age_gender_info.csv")
df_gender.head(3)

,지역,10대미만(여자),10대미만(남자),10대(여자),10대(남자),20대(여자),20대(남자),30대(여자),30대(남자),40대(여자),...,60대(여자),60대(남자),70대(여자),70대(남자),80대(여자),80대(남자),90대(여자),90대(남자),100대(여자),100대(남자)
0,경상북도,0.030158,0.033195,0.056346,0.061360,0.060096,0.067859,0.053433,0.049572,0.083660,...,0.082684,0.063889,0.047717,0.030172,0.029361,0.011211,0.005578,0.001553,0.000234,0.000014
1,경상남도,0.027400,0.026902,0.053257,0.055568,0.064920,0.070618,0.056414,0.057550,0.077092,...,0.087201,0.069562,0.048357,0.033277,0.027361,0.011295,0.004910,0.001086,0.000179,0.000010
2,대전광역시,0.028197,0.029092,0.040490,0.042793,0.060834,0.064247,0.068654,0.066848,0.074667,...,0.088468,0.070261,0.051010,0.037143,0.032455,0.013751,0.006494,0.001740,0.000298,0.000066


In [349]:
df_gender.loc[:,'지역'] = df_gender.loc[:,'지역'].astype('category').cat.codes
df_gender.head(3)

,지역,10대미만(여자),10대미만(남자),10대(여자),10대(남자),20대(여자),20대(남자),30대(여자),30대(남자),40대(여자),...,60대(여자),60대(남자),70대(여자),70대(남자),80대(여자),80대(남자),90대(여자),90대(남자),100대(여자),100대(남자)
0,3,0.030158,0.033195,0.056346,0.061360,0.060096,0.067859,0.053433,0.049572,0.083660,...,0.082684,0.063889,0.047717,0.030172,0.029361,0.011211,0.005578,0.001553,0.000234,0.000014
1,2,0.027400,0.026902,0.053257,0.055568,0.064920,0.070618,0.056414,0.057550,0.077092,...,0.087201,0.069562,0.048357,0.033277,0.027361,0.011295,0.004910,0.001086,0.000179,0.000010
2,6,0.028197,0.029092,0.040490,0.042793,0.060834,0.064247,0.068654,0.066848,0.074667,...,0.088468,0.070261,0.051010,0.037143,0.032455,0.013751,0.006494,0.001740,0.000298,0.000066


# 간단한 모델링

In [83]:
def merge(df_train):
    codes = df_train.단지코드.unique()
    areas = np.sort(df_train.전용면적.unique())

    df_train_edited = pd.DataFrame()
    columns = ['단지코드', '등록차량수', '총세대수', '지역', '공가수','지하철', '버스', '단지내주차면수']

    for order, code in enumerate(codes):
        temp_by_code = df_train.loc[df_train.단지코드==code].reset_index(drop=True)
        ## 원 계열 값 그냥 가져오기 
        df_train_edited.loc[order, columns] = temp_by_code.loc[0, columns]             
        df_train_edited.loc[order, "총임대가구수"] = temp_by_code.전용면적별세대수.sum()

        for area in areas:
            temp_by_code_areas = temp_by_code.loc[temp_by_code.전용면적==area].reset_index(drop=True)

            if temp_by_code_areas.shape[0] !=0:
                df_train_edited.loc[order, f'면적_{int(area)}'] = temp_by_code_areas.전용면적별세대수.sum() / temp_by_code_areas.총세대수[0]
            else:
                df_train_edited.loc[order, f'면적_{int(area)}'] = 0


    df_train_edited["임대비율"] = df_train_edited.총임대가구수 / df_train_edited.총세대수
    df_train_edited["가구당주차면수"] = df_train_edited.단지내주차면수 / df_train_edited.총세대수

    ## gender 정보 병합 
    df_train_edited = pd.merge(df_train_edited, df_gender, left_on= [ "지역"], right_on= ["지역"], how='left')


    return df_train_edited

In [84]:
new_train1=merge(train1)
new_train2=merge(train2)
new_train0=merge(train0)

In [45]:
new_train0

,단지코드,등록차량수,총세대수,지역,공가수,지하철,버스,단지내주차면수,총임대가구수,면적_15,...,60대(여자),60대(남자),70대(여자),70대(남자),80대(여자),80대(남자),90대(여자),90대(남자),100대(여자),100대(남자)
0,C2483,1015.0,900.0,3.0,38.0,0.0,3.0,1425.0,900.0,0.000000,...,0.082684,0.063889,0.047717,0.030172,0.029361,0.011211,0.005578,0.001553,0.000234,0.000014
1,C2515,205.0,545.0,2.0,17.0,0.0,3.0,624.0,545.0,0.000000,...,0.087201,0.069562,0.048357,0.033277,0.027361,0.011295,0.004910,0.001086,0.000179,0.000010
2,C1407,1064.0,1216.0,6.0,13.0,1.0,1.0,1285.0,1216.0,0.000000,...,0.088468,0.070261,0.051010,0.037143,0.032455,0.013751,0.006494,0.001740,0.000298,0.000066
3,C1945,730.0,755.0,1.0,6.0,1.0,3.0,734.0,755.0,0.000000,...,0.074237,0.058419,0.042422,0.032725,0.025136,0.012354,0.005390,0.001707,0.000290,0.000067
4,C1470,553.0,696.0,12.0,14.0,0.0,2.0,645.0,696.0,0.000000,...,0.076636,0.068042,0.051025,0.035748,0.035049,0.012641,0.007223,0.001898,0.000158,0.000013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
418,C2586,57.0,90.0,13.0,7.0,0.0,3.0,66.0,90.0,0.133333,...,0.074248,0.055717,0.047944,0.033054,0.026836,0.011332,0.006832,0.000982,0.000368,0.000082
419,C2035,246.0,492.0,0.0,24.0,0.0,1.0,521.0,492.0,0.000000,...,0.088423,0.070014,0.047732,0.032760,0.033515,0.013027,0.007628,0.001677,0.000319,0.000017
420,C2020,19.0,40.0,7.0,7.0,1.0,2.0,25.0,40.0,0.500000,...,0.109297,0.085294,0.078743,0.053388,0.047908,0.020228,0.008043,0.002240,0.000268,0.000028
421,C2437,16.0,90.0,15.0,12.0,0.0,1.0,30.0,90.0,0.000000,...,0.075390,0.062427,0.041814,0.027566,0.027762,0.011212,0.005386,0.001310,0.000257,0.000037


In [46]:
corr = new_train0.drop(['단지코드'],1,).corr()["등록차량수"].abs().sort_values(ascending=False)
corr

등록차량수        1.000000
단지내주차면수      0.848199
총세대수         0.579290
총임대가구수       0.565676
가구당주차면수      0.476397
면적_70        0.316414
30대(남자)      0.303347
30대(여자)      0.289873
면적_30        0.283405
50대(남자)      0.266140
면적_100       0.260449
면적_20        0.234612
면적_50        0.204213
50대(여자)      0.203076
40대(남자)      0.202237
20대(남자)      0.198262
면적_80        0.196680
면적_60        0.190025
면적_15        0.173557
공가수          0.167420
10대(남자)      0.162642
100대(여자)     0.159708
10대(여자)      0.137323
10대미만(남자)    0.125364
10대미만(여자)    0.124643
지역           0.121607
버스           0.111137
60대(남자)      0.110685
임대비율         0.094771
80대(여자)      0.085312
20대(여자)      0.071159
90대(여자)      0.070871
90대(남자)      0.046605
60대(여자)      0.043460
면적_40        0.027273
70대(여자)      0.026245
40대(여자)      0.022119
100대(남자)     0.016787
70대(남자)      0.015249
지하철          0.010202
80대(남자)      0.003499
Name: 등록차량수, dtype: float64

In [85]:
#등록차량수와 상관도 높은 변수 추출,target값 설정
# 일단 kfold n_split=5로 설정( 더 높이면 결과값이 나아지긴함. 근데 높으면 부작용 없나??)
def result(new_train):
    corr = new_train.drop(['단지코드'],1,).corr()["등록차량수"].abs().sort_values(ascending=False)
    X =new_train.copy()
    X = new_train.loc[:,corr.index[0:20]]
    X.drop(['등록차량수',  ],axis=1, inplace=True)
    feature_names = X.columns.to_list()
    scaler = StandardScaler()
    X = scaler.fit(X).transform(X)## 표준화

    y = new_train.iloc[:,1]
    
    models = {'RF' : RandomForestRegressor(), 'LR': LinearRegression() , 'RD' : Ridge(), 'LS' : Lasso(), 'ET' : ElasticNet(),
          'XGB' : XGBRegressor(), 
          'LGB' : LGB.LGBMRegressor(), 
          'CB' : CatBoostRegressor(logging_level='Silent'), 
          'PLS' : PLSRegression()}

    kfold = KFold(n_splits=5, shuffle = True, random_state=0)
    # n_split : 몇개로 분할할지
    # shuffle : Fold를 나누기 전에 무작위로 섞을지
    # random_state : 나눈 Fold를 그대로 사용할지
    answer = []
    for model in models.keys():
        print(model)
        scores = cross_val_score(models[model] , X,y, cv=kfold, scoring='neg_mean_absolute_error')
        answer.append(scores)

    corss_val_result = pd.DataFrame(answer)
    corss_val_result.index = models.keys()
    corss_val_result['mean'] = corss_val_result.mean(axis=1)
    
    return corss_val_result

    # 피쳐 선택은 나중에 우선 상관도 높은 20개 사용

### train 데이터에만 있는 명목변수값 '이외'로 합쳤을 경우

In [359]:
result(new_train1) ## 

RF
LR
RD
LS
ET
XGB
[21:54:26] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:54:26] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:54:26] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:54:26] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:54:27] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
LGB
CB
PLS


,0,1,2,3,4,mean
RF,-142.266824,-115.602118,-117.716588,-125.107381,-137.495833,-127.637749
LR,-138.266205,-123.986512,-115.955544,-305.571258,-137.426946,-164.241293
RD,-138.489013,-122.805990,-114.497727,-171.260817,-134.400871,-136.290884
LS,-139.022607,-122.426311,-112.999639,-126.188853,-133.433150,-126.814112
ET,-148.258551,-128.015531,-132.230157,-138.529123,-154.945583,-140.395789
XGB,-135.754194,-120.652055,-127.366087,-127.405796,-128.148735,-127.865374
LGB,-139.440021,-104.629473,-126.389329,-118.536883,-130.897990,-123.978739
CB,-124.117019,-104.022496,-118.710956,-128.382245,-131.112544,-121.269052
PLS,-152.850373,-125.449598,-136.717925,-234.390749,-151.480504,-160.177830


### train데이터에만 있는 명목변수값 drop했을 경우

In [49]:
result(new_train2)

RF
LR
RD
LS
ET
XGB
[14:54:06] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:54:06] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:54:07] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:54:07] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:54:07] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:54:07] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:54:07] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
LGB
CB
PLS


,0,1,2,3,4,5,6,mean
RF,-132.542712,-155.990169,-135.890678,-120.265932,-129.276271,-119.086379,-120.092414,-130.449222
LR,-105.872662,-186.104274,-128.834907,-120.436661,-141.486229,-123.190599,-133.240052,-134.166483
RD,-104.322257,-174.714535,-127.862516,-120.294981,-141.098751,-121.921280,-133.439708,-131.950575
LS,-102.686241,-169.914470,-127.198196,-119.766336,-138.842686,-119.616101,-133.534159,-130.222599
ET,-109.229594,-149.909165,-141.540200,-129.416702,-137.115560,-140.406571,-157.326887,-137.849240
XGB,-126.905228,-138.689414,-138.893091,-128.735630,-137.861739,-111.803654,-108.531805,-127.345795
LGB,-119.574512,-135.051309,-137.163023,-138.641502,-152.232281,-116.362441,-117.955434,-130.997215
CB,-117.074119,-133.123415,-122.362201,-128.799740,-134.268754,-102.928401,-114.521431,-121.868294
PLS,-110.924296,-157.740179,-157.289405,-127.921908,-153.957562,-146.173496,-156.496251,-144.357585


### train데이터에만 있는 명목변수값 그대로 두었을 때

In [360]:
result(new_train0)

RF
LR
RD
LS
ET
XGB
[21:54:50] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:54:50] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:54:50] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:54:50] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:54:51] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
LGB
CB
PLS


,0,1,2,3,4,mean
RF,-140.396471,-115.385647,-118.610118,-117.823690,-142.135238,-126.870233
LR,-139.461653,-123.771701,-110.546918,-305.796352,-133.663121,-162.647949
RD,-139.711602,-123.797972,-109.312895,-167.184049,-129.127590,-133.826822
LS,-139.727726,-122.800273,-109.729598,-124.081823,-128.791766,-125.026237
ET,-142.930596,-130.218128,-130.925150,-136.228974,-154.414468,-138.943463
XGB,-130.340615,-117.492071,-124.839510,-123.584120,-135.148315,-126.280926
LGB,-135.945600,-102.366766,-127.295549,-124.314122,-127.279969,-123.440401
CB,-121.104666,-101.221249,-112.561215,-130.265161,-130.501529,-119.130764
PLS,-142.226294,-127.350508,-136.696227,-300.653549,-153.963912,-172.178098


In [ ]:
model = RF = LGBMRegressor(n_jobs=-1, random_state=300, max_depth =12,	min_samples_leaf =2,	min_samples_split=2,	n_estimators=200)
model.fit(X, y)

X_test = df_test_edited[feature_names]

pred = RF.predict(X_test)
submission = dataset[files[3]]

submission['num'] = pred
submission.to_csv('baseline.csv', index=False)


LGB,CB 모델이 좋은 성능을 보이고, train데이터에만 있는 명목변수값을 그냥 두었을 때 가장 성능이 좋음. \
다른 조건을 변경하고 결과를 비교할 때 LGB,CB를 중심으로 결과값 비교해봄\
전용면적을 10단위로 나누었는데, 5단위로 해보니 결과값이 더 안좋아짐.\
n_splits를 7까지 높였을 때 CB 결괏값이 -115까지 낮아짐.\
